In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
def make_soup(filepath):
#     html = requests.get(file)
    return BeautifulSoup(open(filepath), "lxml")

In [3]:
file = 'jira_your_work.htm'

In [4]:
test = make_soup(file)

In [5]:
# len(test.find_all('script'))

In [6]:
# test.find_all('script')[0]

In [7]:
# type(test.find_all('script')[1])

In [8]:
# test.find_all('script')[12].contents[0]

In [9]:
test.find_all(attrs={"data-test-id":r"global-pages.home.ui.tab-container.tab.item-list.item-link#issue-656551"})

[<a class="sc-eQubti htSXGQ" data-test-id="global-pages.home.ui.tab-container.tab.item-list.item-link#issue-656551" href="/browse/BUS-4411" target="_self"><div class="sc-hycuFf hwReiR"><img class="sc-iEGEut fDdIpv" src="https://justeattakeaway.atlassian.net/secure/viewavatar?size=medium&amp;avatarId=10318&amp;avatarType=issuetype"/></div><span class="sc-fXUGxx bzdxcH"><span class="sc-iTlrqL biRwIE">Scope work for Conversocial data reports</span><small class="sc-fcLdzD fHVdRH"><span>BUS-4411</span><span>·</span><span>Business Intelligence</span></small></span><span class="sc-lmwPYW dJlFHW">Open</span><div class="sc-hQrNYi cAvKoV"></div></a>]

In [10]:
attr_value_regex = re.compile(r"global-pages.home.ui.tab-container.tab.item-list.item-link#issue-[0-9]{6}")

In [11]:
all_tickets = test.find_all(attrs={"data-test-id":attr_value_regex})

In [12]:
len(all_tickets)

17

In [13]:
all_tickets[0]

<a class="sc-eQubti htSXGQ" data-test-id="global-pages.home.ui.tab-container.tab.item-list.item-link#issue-560827" href="/browse/PA-1194" target="_self"><div class="sc-hycuFf hwReiR"><img class="sc-iEGEut fDdIpv" src="https://justeattakeaway.atlassian.net/secure/viewavatar?size=medium&amp;avatarId=10300&amp;avatarType=issuetype"/></div><span class="sc-fXUGxx bzdxcH"><span class="sc-iTlrqL biRwIE">Analysis into typing time in Zendesk</span><small class="sc-fcLdzD fHVdRH"><span>PA-1194</span><span>·</span><span>Product Analytics</span></small></span><span class="sc-lmwPYW dJlFHW">On Hold (Action)</span><div class="sc-hQrNYi cAvKoV"></div></a>

In [14]:
all_tickets[1]

<a class="sc-eQubti htSXGQ" data-test-id="global-pages.home.ui.tab-container.tab.item-list.item-link#issue-647205" href="/browse/BUS-4340" target="_self"><div class="sc-hycuFf hwReiR"><img class="sc-iEGEut fDdIpv" src="https://justeattakeaway.atlassian.net/secure/viewavatar?size=medium&amp;avatarId=10318&amp;avatarType=issuetype"/></div><span class="sc-fXUGxx bzdxcH"><span class="sc-iTlrqL biRwIE">Add re-open rate view to async contacts dashboard</span><small class="sc-fcLdzD fHVdRH"><span>BUS-4340</span><span>·</span><span>Business Intelligence</span></small></span><span class="sc-lmwPYW dJlFHW">Reopened</span><div class="sc-hQrNYi cAvKoV"></div></a>

In [15]:
type(all_tickets[0])

bs4.element.Tag

In [16]:
all_tickets[0].find_all('span')

[<span class="sc-fXUGxx bzdxcH"><span class="sc-iTlrqL biRwIE">Analysis into typing time in Zendesk</span><small class="sc-fcLdzD fHVdRH"><span>PA-1194</span><span>·</span><span>Product Analytics</span></small></span>,
 <span class="sc-iTlrqL biRwIE">Analysis into typing time in Zendesk</span>,
 <span>PA-1194</span>,
 <span>·</span>,
 <span>Product Analytics</span>,
 <span class="sc-lmwPYW dJlFHW">On Hold (Action)</span>]

In [27]:
for s in all_tickets[0].find_all('span'):
    print(s.contents)
    print("------")

[<span class="sc-iTlrqL biRwIE">Analysis into typing time in Zendesk</span>, <small class="sc-fcLdzD fHVdRH"><span>PA-1194</span><span>·</span><span>Product Analytics</span></small>]
------
['Analysis into typing time in Zendesk']
------
['PA-1194']
------
['·']
------
['Product Analytics']
------
['On Hold (Action)']
------


In [25]:
for s in all_tickets[1].find_all('span'):
    print(s.contents)
    print("------")

[<span class="sc-iTlrqL biRwIE">Add re-open rate view to async contacts dashboard</span>, <small class="sc-fcLdzD fHVdRH"><span>BUS-4340</span><span>·</span><span>Business Intelligence</span></small>]
------
['Add re-open rate view to async contacts dashboard']
------
['BUS-4340']
------
['·']
------
['Business Intelligence']
------
['Reopened']
------


In [22]:
for s in all_tickets[2].find_all('span'):
    print(s.contents)
    print("------")

[<span class="sc-iTlrqL biRwIE">Interim snapshot numbers for offline bad items queue</span>, <small class="sc-fcLdzD fHVdRH"><span>BUS-4284</span><span>·</span><span>Business Intelligence</span></small>]
------
['Interim snapshot numbers for offline bad items queue']
------
['BUS-4284']
------
['·']
------
['Business Intelligence']
------
['Pending']
------
